# Module 07: Interacting with Databases
This notebook demonstrates how to query SQL db.

## What we'll learn:
- SQLAlchemy Engine
- Listing Tables
- Getting Table Schema
- Executing SQL
- Text2SQL Agent

### Setup

In [1]:
from typing import List, Any
import sqlalchemy
from sqlalchemy.engine.base import Engine
from sqlalchemy import text, create_engine
from dotenv import load_dotenv
import pandas as pd

from lib.messages import BaseMessage
from lib.tooling import tool
from lib.agents import Agent

In [2]:
load_dotenv()

True

In [3]:
DB_ENGINE = create_engine(f"sqlite:///sales.db")

## SQL Toolkit

In [4]:
@tool
def list_tables_tool() -> List[str]:
    """
    List all tables in database
    """
    inspector = sqlalchemy.inspect(DB_ENGINE)

    return inspector.get_table_names()

In [5]:
@tool
def get_table_schema_tool(table_name:str) -> List[str]:
    """
    Get schema information about a table. Returns a list of dictionaries.
    - name is the column name
    - type is the column type
    - nullable is whether the column is nullable or not
    - default is the default value of the column
    - primary_key is whether the column is a primary key or not

    Args:
        table_name (str): Table name
    """
    inspector = sqlalchemy.inspect(DB_ENGINE)

    return str(inspector.get_columns(table_name))


In [6]:
@tool
def execute_sql_tool(query:str) -> Any:
    """
    Execute SQL query and return result. 
    This will automatically connect to the database and execute the query.
    However, if the query is not valid, an error will be raised

    Args:
        query (str): SQL query
    """
    with DB_ENGINE.begin() as connection:
        answer = connection.execute(text(query)).fetchall()

    return str(answer)

In [7]:
tables = list_tables_tool()
tables

['sales']

In [8]:
schemas = {
    table: get_table_schema_tool(table_name=table) 
    for table in tables
}
schemas

{'sales': "[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'primary_key': 1}, {'name': 'transaction_date', 'type': DATE(), 'nullable': False, 'default': None, 'primary_key': 0}, {'name': 'model', 'type': VARCHAR(length=50), 'nullable': False, 'default': None, 'primary_key': 0}, {'name': 'price', 'type': FLOAT(), 'nullable': False, 'default': None, 'primary_key': 0}, {'name': 'quantity', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'customer_id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}]"}

In [9]:
sql = f"SELECT * FROM {tables[0]} LIMIT 10"
result = execute_sql_tool(query=sql)
result

"[(1, '2024-05-22', 'Dell XPS 15', 1308.04, 4, 1037), (2, '2024-06-07', 'Dell Inspiron 15', 2399.23, 4, 1044), (3, '2024-12-07', 'Dell XPS 13', 2481.61, 2, 1013), (4, '2024-06-21', 'Dell G5 15', 505.98, 4, 1054), (5, '2024-01-04', 'Dell Inspiron 14', 1429.78, 1, 1097), (6, '2024-04-23', 'Dell Inspiron 14', 1141.06, 2, 1074), (7, '2024-06-18', 'Dell Latitude 7310', 1740.03, 2, 1094), (8, '2024-03-12', 'Dell XPS 15', 521.49, 3, 1019), (9, '2024-02-10', 'Dell Latitude 7410', 545.36, 4, 1003), (10, '2024-12-17', 'Dell Inspiron 14', 668.95, 3, 1010)]"

In [10]:
tools = [list_tables_tool, get_table_schema_tool, execute_sql_tool]

In [11]:
sql_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are a Sr. SQL developer tasked with generating SQL queries. Perform the following steps:\n"
        "First, find out the appropriate table name based on all tables. "
        "Then get the table's schema to understand the columns. "
        "With the table name and the schema, generate the ANSI SQL query you think is applicable to the user question. "
        "Finally, use a tool to execute the above SQL query and output the result based on the user question."
    ),
    tools=tools,
)

In [12]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [13]:
run1 = sql_agent.invoke(
    query="How many Dell XPS 15 were sold?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a Sr. SQL developer tasked with generating SQL queries. Perform the following steps:
First, find out the appropriate table name based on all tables. Then get the table's schema to understand the columns. With the table name and the schema, generate the ANSI SQL query you think is applicable to the user question. Finally, use a tool to execute the above SQL query and output the result based on the user question., tool_calls = None)
 -> (role = user, content = How many Dell XPS 15 were sold?, tool_c